# Replicate Paper Results: 1D and 2D Spectroscopy Data Processing

This notebook loads and plots both 1D and 2D spectroscopy data from pickle files to replicate paper results.

# Generalized Functions for Data Loading

In [1]:
# =============================
# IMPORTS
# =============================
import os
import sys
import gzip
import pickle
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

### Project-specific imports
from config.paths import DATA_DIR, FIGURES_DIR
from qspectro2d.spectroscopy.post_processing import extend_and_plot_results
from qspectro2d.visualization.plotting import plot_2d_el_field, plot_fixed_tau_t
from config import mpl_tex_settings

🖥️  Forcing Agg backend to simulate headless HPC environment


In [ ]:
# =============================
# GENERALIZED DATA LOADING FUNCTIONS
# =============================

def find_latest_file(data_subdir: str, file_pattern: str = "*.pkl") -> Path:
    """Find the most recent file matching pattern in a data subdirectory.
    
    Args:
        data_subdir: Subdirectory within DATA_DIR (e.g., '1d_spectroscopy', 'raw/2d_spectroscopy')
        file_pattern: Glob pattern for file matching
    
    Returns:
        Path to the latest file or None if not found
    """
    data_dir = DATA_DIR / data_subdir
    
    if not data_dir.exists():
        print(f"❌ Data directory does not exist: {data_dir}")
        return None
    
    # Look for files matching the pattern
    files = list(data_dir.glob(file_pattern))
    
    # For 2D data, also look for compressed files
    if "2d_spectroscopy" in data_subdir:
        files.extend(list(data_dir.glob("*.pkl.gz")))
    
    if not files:
        print(f"❌ No files matching '{file_pattern}' found in {data_dir}")
        return None
    
    # Get the most recent file
    latest_file = max(files, key=lambda p: p.stat().st_mtime)
    print(f"✅ Found latest file: {latest_file.name}")
    return latest_file


def load_pickle_file(filepath: Path) -> dict:
    """Load data from pickle file (supports both .pkl and .pkl.gz).
    
    Args:
        filepath: Path to the pickle file
    
    Returns:
        Dictionary containing the loaded data or None if error
    """
    print(f"Loading data from: {filepath.name}")
    
    try:
        if filepath.suffix == ".gz":
            # Handle compressed pickle files
            with gzip.open(filepath, "rb") as f:
                data = pickle.load(f)
        else:
            # Handle regular pickle files
            with open(filepath, "rb") as f:
                data = pickle.load(f)
        
        print(f"✅ Data loaded successfully!")
        return data
    
    except Exception as e:
        print(f"❌ Error loading file: {e}")
        return None


def create_output_directory(subdir: str) -> Path:
    """Create output directory for figures.
    
    Args:
        subdir: Subdirectory name within FIGURES_DIR
    
    Returns:
        Path to the created output directory
    """
    output_dir = FIGURES_DIR / subdir
    os.makedirs(output_dir, exist_ok=True)
    return output_dir


from typing import Dict, Any
def average_data_1d(data_dir: str, file_pattern: str = "*.pkl", output_filename: str = "averaged_data.pkl") -> Dict[str, Any]:
    """Average data from multiple pickle files in a directory.
    
    Args:
        data_dir: Directory containing pickle files to process
        file_pattern: Pattern to match files (e.g., "*.pkl", "1d_data_*.pkl")
        output_filename: Name of the output file to save averaged data
        
    Returns:
        Dictionary containing the averaged data
    """
    # Convert to Path object for consistency
    data_path = Path(data_dir)
    
    # Get all matching pickle files in the directory
    pkl_files = list(data_path.glob(file_pattern))
    
    # Also look for compressed files if applicable
    if "pkl" in file_pattern:
        compressed_pattern = file_pattern.replace(".pkl", ".pkl.gz")
        pkl_files.extend(list(data_path.glob(compressed_pattern)))
    
    if not pkl_files:
        print(f"❌ No files matching '{file_pattern}' found in {data_dir}")
        return None
    
    pkl_files.sort()  # Sort for consistent processing
    print(f"Found {len(pkl_files)} pickle files to average.")
    
    # Initialize variables to store accumulated data
    sum_data_avg = None
    total_n_freqs = 0
    first_file_data = None
    
    # Process each file and accumulate data
    for i, file_path in enumerate(pkl_files):
        # Use the existing load_pickle_file function
        loaded_data = load_pickle_file(file_path)
        
        if loaded_data is not None:
            # Store data from the first file to use as template
            if i == 0:
                first_file_data = loaded_data
                # Initialize sum_data_avg with the right shape
                sum_data_avg = loaded_data['data_avg'].copy()
            else:
                # Add data_avg from current file
                sum_data_avg += loaded_data['data_avg']
            
            # Accumulate n_freqs
            total_n_freqs += loaded_data['n_freqs']
            
            print(f"✅ Processed file {i+1}/{len(pkl_files)}")
        else:
            print(f"❌ Skipping file due to load error: {file_path.name}")
    
    # Check if we have valid data to average
    if first_file_data is None or sum_data_avg is None:
        print("❌ No valid data files were processed.")
        return None
        
    # Count successfully processed files (might be less than total if some failed)
    processed_count = sum(1 for f in pkl_files if load_pickle_file(f) is not None)
    
    if processed_count == 0:
        print("❌ No files were successfully processed.")
        return None
    
    # Create averaged data dictionary
    averaged_data = {
        't_det_vals': first_file_data['t_det_vals'],
        'data_avg': sum_data_avg / processed_count,  # Average the accumulated data
        'tau_coh': first_file_data['tau_coh'],
        'T_wait': first_file_data['T_wait'],
        'system': first_file_data['system'],
        'n_freqs': total_n_freqs
    }
    
    # Save averaged data to output file
    output_path = data_path / output_filename
    with open(output_path, 'wb') as f:
        pickle.dump(averaged_data, f)
    
    print(f"✅ Averaged data saved to: {output_path}")
    print(f"  - Averaged across {processed_count} files")
    print(f"  - Total frequencies: {total_n_freqs}")
    
    return averaged_data

def average_data_2d(data_dir: str, file_pattern: str = "*d.pkl", output_filename: str = "averaged_2d_data.pkl") -> Dict[str, Any]:
    """Average 2D spectroscopy data from multiple pickle files in a directory.
    
    Args:
        data_dir: Directory containing pickle files to process
        file_pattern: Pattern to match files (e.g., "*d.pkl", "2d_data_*.pkl")
        output_filename: Name of the output file to save averaged data
        
    Returns:
        Dictionary containing the averaged data
    """
    # Convert to Path object for consistency
    data_path = Path(data_dir)
    
    # Get all matching pickle files in the directory
    pkl_files = list(data_path.glob(file_pattern))
    
    # Also look for compressed files if applicable
    if "pkl" in file_pattern:
        compressed_pattern = file_pattern.replace(".pkl", ".pkl.gz")
        pkl_files.extend(list(data_path.glob(compressed_pattern)))
    
    if not pkl_files:
        print(f"❌ No files matching '{file_pattern}' found in {data_dir}")
        return None
    
    pkl_files.sort()  # Sort for consistent processing
    print(f"Found {len(pkl_files)} pickle files to average.")
    
    # Initialize variables to store accumulated data
    sum_two_d_datas = None
    total_n_freqs = 0
    first_file_data = None
    processed_count = 0
    
    # Process each file and accumulate data
    for i, file_path in enumerate(pkl_files):
        # Use the existing load_pickle_file function
        loaded_data = load_pickle_file(file_path)
        
        if loaded_data is not None:
            # Store data from the first file to use as template
            if i == 0:
                first_file_data = loaded_data
                # Initialize sum_two_d_datas with the right shape AND dtype
                # For 2D data: two_d_datas is a list of arrays, one for each T_wait time
                # Use astype(complex) to ensure we preserve complex values
                sum_two_d_datas = [arr.copy().astype(complex) for arr in loaded_data['two_d_datas']]
            else:
                # Add corresponding 2D arrays for each T_wait time
                for j, arr in enumerate(loaded_data['two_d_datas']):
                    sum_two_d_datas[j] += arr.astype(complex)
            
            # Accumulate n_freqs
            total_n_freqs += loaded_data['n_freqs']
            processed_count += 1
            
            print(f"✅ Processed file {i+1}/{len(pkl_files)}: {file_path.name}")
        else:
            print(f"❌ Skipping file due to load error: {file_path.name}")
    
    # Check if we have valid data to average
    if first_file_data is None or sum_two_d_datas is None:
        print("❌ No valid data files were processed.")
        return None
        
    if processed_count == 0:
        print("❌ No files were successfully processed.")
        return None
    
    # Create averaged data dictionary
    averaged_data = {
        'times': first_file_data['times'],
        'times_T': first_file_data['times_T'],
        'system': first_file_data['system'],
        # Calculate average for each 2D array in the list
        'two_d_datas': [arr / processed_count for arr in sum_two_d_datas],
        'n_times_T': first_file_data['n_times_T'],
        'n_phases': first_file_data['n_phases'],
        'n_freqs': total_n_freqs  # Sum of all n_freqs values
    }
    
    # Save averaged data to output file
    output_path = data_path / output_filename
    with open(output_path, 'wb') as f:
        pickle.dump(averaged_data, f)
    
    print(f"✅ Averaged 2D data saved to: {output_path}")
    print(f"  - Averaged across {processed_count} files")
    print(f"  - Total frequencies: {total_n_freqs}")
    print(f"  - Number of T wait times: {len(averaged_data['two_d_datas'])}")
    print(f"  - Shape of each 2D array: {averaged_data['two_d_datas'][0].shape}")
    
    return averaged_data


In [ ]:
# =============================
# AVERAGE SIMULATION DATA FROM MULTIPLE PICKLE FILES
# =============================

# Example usage:
# Set data_dir to the directory containing pickle files to average
# Use pathlib for path operations
data_dir = DATA_DIR / "1d_spectroscopy" / "inhomogeneity"
averaged_data = average_data_1d(
    data_dir,
    file_pattern="1d_data_*.pkl",  # Optional: specify pattern to match specific files
    output_filename="1d_data_averaged.pkl"
)

# You can then use the averaged data for plotting or further analysis
# For example:
if averaged_data is not None:
    plot_fixed_tau_t(
        averaged_data['t_det_vals'], 
        averaged_data['data_avg'],
        tau_coh=averaged_data['tau_coh'],
        T_wait=averaged_data['T_wait'],
        n_freqs=averaged_data['n_freqs']
    )


# 1. Process 1D Spectroscopy Data (Fixed τ, T)

In [ ]:
# =============================
# LOAD AND PLOT 1D SPECTROSCOPY DATA
# =============================

### Find and load 1D data
file_path_1d = find_latest_file("1d_spectroscopy/inhomogeneity", "1d_data_*.pkl")

if file_path_1d is not None:
    loaded_data = load_pickle_file(file_path_1d)
    
    if loaded_data is not None:
        # Extract 1D data
        t_det_vals = loaded_data['t_det_vals']
        data_avg   = loaded_data['data_avg']
        tau_coh    = loaded_data['tau_coh']
        T_wait     = loaded_data['T_wait']
        system     = loaded_data['system']
        n_freqs  = loaded_data['n_freqs']
        
        ### Display data information
        print(f"  τ_coh = {tau_coh:.2f} fs")
        print(f"  T_wait = {T_wait:.2f} fs")
        print(f"  t_det_vals: {len(t_det_vals)} points from {t_det_vals[0]:.2f} to {t_det_vals[-1]:.2f} fs")
        print(f"  data_avg shape: {data_avg.shape}")
        print(f"  ω_ats: {n_freqs} frequencies")
        print(f"  System parameters: {system}")
        
        ### Prepare plotting arguments
        output_dir = create_output_directory("1d_spectroscopy/inhomogeneity")
        kwargs = {
            'tau_coh': tau_coh,
            'T_wait': T_wait,
            'n_freqs': n_freqs,
            'envelope': system.pulse_type,
            'RWA': system.RWA_SL,
            'Solver': system.ODE_Solver,
            'function': "E_{k_s}", # ONLY IF THE DATA comes from "parallel_compute_1d_E_with_inhomogeneity",
            'show': False  # Don't show the plot yet, we'll save it first
        }
        
        ### Generate 1D plot
        print("📊 Generating 1D spectroscopy plot...")
        # Plot and get the figure object
        fig = plot_fixed_tau_t(t_det_vals, data_avg, **kwargs)

        # Save the figure with a descriptive filename
        output_path = output_dir / f"1d_spectroscopy_{file_path_1d.stem}.png"
        fig.savefig(output_path, dpi=300, bbox_inches='tight')
        print(f"✅ 1D plot saved to: {output_path}")
        
        # Now display the plot
        plt.show()
        
    else:
        print("❌ Failed to load 1D spectroscopy data.")
else:
    print("   Please run the 1D spectroscopy calculation first to generate data.")


# 2. Process 2D Spectroscopy Data

In [ ]:
# =============================
# AVERAGE SIMULATION DATA FROM MULTIPLE PICKLE FILES
# =============================

# Example usage:
# Set data_dir to the directory containing pickle files to average
# Use pathlib for path operations
data_dir = DATA_DIR / "2d_spectroscopy" / "new_echo_signal" / "900fs"
averaged_data = average_data_2d(
    data_dir,
    file_pattern="2d_data_*.pkl",  # Optional: specify pattern to match specific files
    output_filename="2d_data_averaged.pkl"
)

In [ ]:
# =============================
# LOAD AND PLOT 2D SPECTROSCOPY DATA
# =============================

### Find and load 2D data
file_path_2d = find_latest_file("2d_spectroscopy/new_echo_signal/600fs/", "*.pkl")

if file_path_2d is not None:
    loaded_data = load_pickle_file(file_path_2d)
    
    if loaded_data is not None:
        # Extract 2D data
        two_d_datas = loaded_data["two_d_datas"]
        times_T     = loaded_data["times_T"]
        times       = loaded_data["times"]
        system_data = loaded_data["system"]
        
        ### Display data information
        print(f"  Loaded {len(two_d_datas)} 2D datasets")
        print(f"  System data: {system_data}")
        
        ### Create output directory
        output_dir = create_output_directory("2d_spectroscopy/extended")
        
        from qspectro2d.spectroscopy.calculations import get_tau_cohs_and_t_dets_for_T_wait
        t_det_vals, tau_coh_vals = get_tau_cohs_and_t_dets_for_T_wait(times, times_T[0])
        """
        plot_2d_el_field(
            (t_det_vals, tau_coh_vals, two_d_datas[0]),
            save=True,  # CHANGE TO False for no plotting the Time domain
            output_dir=output_dir,
            system=system_data,
            use_custom_colormap=True,
        )
        """
        ### Plot each component type
        spectral_components_to_plot = ["imag", "abs", "real", "phase"]
        
        for component in spectral_components_to_plot:
            print(f"📊 Plotting {component} component...")
            
            plot_args = {
                "domain": "freq",
                "type": component,
                "save": True,
                "output_dir": output_dir,
                "use_custom_colormap": True,
                "section": (1.5, 1.7, 1.5, 1.7),  # Plot the first section
                "system": system_data,
            }
            
            try:
                extend_and_plot_results(
                    two_d_datas,
                    times_T=times_T,
                    times=times,
                    extend_for=(1, 2.3),
                    **plot_args,
                )
                print(f"✅ {component} plot completed!")
                
            except Exception as e:
                print(f"❌ Error plotting {component}: {e}")
        
        print(f"🎯 All 2D plots saved to: {output_dir}")
        
    else:
        print("❌ Failed to load 2D spectroscopy data.")
else:
    print("   Please run the 2D spectroscopy calculation first to generate data.")